In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [4]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [5]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
import xgboost as xgb
from sklearn.ensemble import BaggingRegressor
import numpy as np 
import pandas as pd 
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
import seaborn as sns
from sklearn.metrics import accuracy_score

In [6]:
data = pd.read_csv('US_WeatherEvents_2016-2019.csv')

In [7]:
data.isna().sum()

EventId               0
Type                  0
Severity              0
StartTime(UTC)        0
EndTime(UTC)          0
TimeZone              0
AirportCode           0
LocationLat           0
LocationLng           0
City               9087
County                0
State                 0
ZipCode           38685
dtype: int64

In [8]:
data = data.fillna(data.median())

In [9]:
data.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [10]:
table_1 = data.groupby(['City','Type']).count()

In [11]:
table_1 = table_1.reset_index()

In [12]:
table_1

,City,Type,EventId,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,County,State,ZipCode
0,Abbeville,Cold,23,23,23,23,23,23,23,23,23,23,23
1,Abbeville,Fog,920,920,920,920,920,920,920,920,920,920,920
2,Abbeville,Precipitation,49,49,49,49,49,49,49,49,49,49,49
3,Abbeville,Rain,2473,2473,2473,2473,2473,2473,2473,2473,2473,2473,2473
4,Abbeville,Snow,6,6,6,6,6,6,6,6,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9744,Zionsville,Fog,488,488,488,488,488,488,488,488,488,488,488
9745,Zionsville,Precipitation,32,32,32,32,32,32,32,32,32,32,32
9746,Zionsville,Rain,2514,2514,2514,2514,2514,2514,2514,2514,2514,2514,2514
9747,Zionsville,Snow,479,479,479,479,479,479,479,479,479,479,479


Here I collect the number of types according to cities and then I will merge them with main dataframe.

In [13]:
list_of_dic = []
label = table_1.City[0]
list_val = {}
for index,row in table_1.iterrows():   
    if row['City'] != label:
        label = row['City']
        list_of_dic.append(list_val)
        list_val = {}
        list_val['City'] = row['City']
        list_val[row['Type']] = row['EventId']
       
    else:
        list_val['City'] = row['City']
        list_val[row['Type']] = row['EventId']
        
    
    
    

In [14]:
df = pd.DataFrame()
for dic in list_of_dic:
    df = df.append(dic, ignore_index=True)
    


In [15]:
df = df.fillna(0)

In [16]:
data = data.merge(df,on = 'City')

In [17]:
len(data)

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,Cold,Fog,Precipitation,Rain,Snow,Storm,Hail
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,18.0,61.0,3.0,1063.0,535.0,41.0,0.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,18.0,61.0,3.0,1063.0,535.0,41.0,0.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,18.0,61.0,3.0,1063.0,535.0,41.0,0.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,18.0,61.0,3.0,1063.0,535.0,41.0,0.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,18.0,61.0,3.0,1063.0,535.0,41.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5047183,W-5058448,Storm,Severe,2019-11-24 13:30:00,2019-11-24 13:50:00,US/Mountain,KARL,41.6000,-106.2100,Rock River,Carbon,WY,82083.0,105.0,298.0,10.0,487.0,735.0,603.0,0.0
5047184,W-5058449,Storm,Severe,2019-11-24 14:10:00,2019-11-24 14:30:00,US/Mountain,KARL,41.6000,-106.2100,Rock River,Carbon,WY,82083.0,105.0,298.0,10.0,487.0,735.0,603.0,0.0
5047185,W-5058450,Storm,Severe,2019-11-24 14:50:00,2019-11-24 20:30:00,US/Mountain,KARL,41.6000,-106.2100,Rock River,Carbon,WY,82083.0,105.0,298.0,10.0,487.0,735.0,603.0,0.0
5047186,W-5058451,Storm,Severe,2019-11-24 21:10:00,2019-11-24 21:30:00,US/Mountain,KARL,41.6000,-106.2100,Rock River,Carbon,WY,82083.0,105.0,298.0,10.0,487.0,735.0,603.0,0.0


In [18]:
data['StartTime(UTC)'] = pd.to_datetime(data['StartTime(UTC)'])
data['EndTime(UTC)'] = pd.to_datetime(data['EndTime(UTC)'])

data['Start_year'] = data['StartTime(UTC)'].dt.year
data['Start_month'] = data['StartTime(UTC)'].dt.month
data['Start_week'] = data['StartTime(UTC)'].dt.week
data['Start_weekday'] = data['StartTime(UTC)'].dt.weekday
data['Start_day'] = data['StartTime(UTC)'].dt.day

data['end_year'] = data['EndTime(UTC)'].dt.year
data['end_month'] = data['EndTime(UTC)'].dt.month
data['end_week'] = data['EndTime(UTC)'].dt.week
data['end_weekday'] = data['EndTime(UTC)'].dt.weekday
data['end_day'] = data['EndTime(UTC)'].dt.day


In [ ]:
X = data.drop(['Type','StartTime(UTC)','EndTime(UTC)'],axis = 1).head(1000)
y = data.Type.head(1000)

In [ ]:
y

In [ ]:
le = preprocessing.LabelEncoder()
for name in X.columns:
    if X[name].dtypes == "O":
        print(name)
        X[name] = X[name].astype(str)
        le.fit(X[name])
        X[name] = le.transform(X[name])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
clf = RandomForestClassifier(n_estimators = 400,min_samples_split = 2,min_samples_leaf = 1,max_features= 'sqrt',max_depth =None,bootstrap= False)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

In [ ]:
X

In [ ]:
import seaborn as sns
ax = sns.barplot(x=clf.feature_importances_, y=X.columns)

In [ ]:
accuracy_score(predictions,y_test)